In [4]:
import pandas as pd

# Load the CSV file
kedb_data = pd.read_csv('kedb.csv')

In [5]:
kedb_data.head()

,EventTypeName,Cluster no,Message,Timestamp,Status,Processed_Message
0,SAM Warning,2,"Component ""Worker Process Statistic Monitor (C...",2025-02-18 00:15:31,warning,Component Worker Process Statistic Monitor CCM...
1,SAM Warning,2,"Component ""Worker Process Statistic Monitor (S...",2025-02-18 00:15:31,warning,Component Worker Process Statistic Monitor SMS...
2,SAM Warning,2,"Component ""Worker Process Statistic Monitor (I...",2025-02-18 00:15:31,warning,Component Worker Process Statistic Monitor ISM...
3,SAM Warning,2,"Component ""Active Directory Domain Services"" f...",2025-02-18 00:15:31,warning,Component Active Directory Domain Services app...
4,SAM Warning,2,"Application ""NOC Infra - Active Directory Doma...",2025-02-18 00:15:31,warning,Application NOC Infra Active Directory Domain ...


In [6]:
# Convert the Timestamp column to datetime format
kedb_data['Timestamp'] = pd.to_datetime(kedb_data['Timestamp'])


In [11]:
def query_processed_message(keyword=None, status=None, cluster=None):
    """
    Query the Processed_Message column based on keyword, status, or cluster.
    """
    filtered_data = kedb_data
    if keyword:
        filtered_data = filtered_data[filtered_data['Processed_Message'].str.contains(keyword, case=False, na=False)]
    if status:
        filtered_data = filtered_data[filtered_data['Status'].str.lower() == status.lower()]
    if cluster:
        filtered_data = filtered_data[filtered_data['Cluster no'] == cluster]

    return filtered_data[['Timestamp', 'Cluster no', 'Status', 'Processed_Message']]



In [12]:
def analyze_processed_messages():
    """
    Analyze the Processed_Message column to provide insights.
    """
    most_common = kedb_data['Processed_Message'].value_counts().head(5)
    status_counts = kedb_data['Status'].value_counts()

    return {
        'Most Common Messages': most_common,
        'Status Distribution': status_counts
    }



In [13]:
def provide_suggestions():
    """
    Suggest actions based on common patterns in Processed_Message.
    """
    suggestions = []
    
    if 'Active Directory' in ' '.join(kedb_data['Processed_Message'].tolist()):
        suggestions.append("Investigate Active Directory processes.")
    
    if kedb_data['Status'].str.contains('critical', case=False, na=False).any():
        suggestions.append("Address critical issues immediately.")
    
    if kedb_data['Cluster no'].value_counts().max() > 10:
        suggestions.append("Check clusters with high event frequency.")

    return suggestions



In [14]:
def chatbot_response(user_query):
    """
    Process user queries and search the KEDB for relevant results.
    """
    # Parse the user query for keywords, status, or cluster
    keywords = [word for word in user_query.split() if len(word) > 3]
    status = None
    cluster = None

    # Attempt to identify status or cluster in the query
    if "warning" in user_query.lower():
        status = "warning"
    elif "critical" in user_query.lower():
        status = "critical"

    if "cluster" in user_query.lower():
        try:
            cluster = int([word for word in user_query.split() if word.isdigit()][0])
        except IndexError:
            cluster = None

    # Search the KEDB
    results = query_processed_message(keyword=" ".join(keywords), status=status, cluster=cluster)

    # Format the response
    if not results.empty:
        return results.to_dict('records')
    else:
        return "No matching records found in the KEDB."


In [21]:
if __name__ == "__main__":
    # Example chatbot interaction
    user_query = "Show me warning related ISNOCPRIVAPP0"
    print("Chatbot Response:")
    print(chatbot_response(user_query))

Chatbot Response:
No matching records found in the KEDB.


In [12]:
def search_csv(query, df):
    # Convert the query to lowercase for case-insensitive search
    query = query.lower()
    
    # Search the Processed_Message column for the query
    results = df[df['Processed_Message'].str.lower().str.contains(query)]
    
    if not results.empty:
        return results['Processed_Message'].tolist()
    else:
        return None

In [17]:
def chatbot(query):
    # Search the CSV first
    csv_results = search_csv(query, df)
    
    if csv_results:
        return "From CSV: " + ", ".join(csv_results)
    else:
        # If not found in CSV, use OpenAI GPT
        openai_answer = get_openai_answer(query)
        return "From OpenAI GPT: " + openai_answer

In [18]:
# Example queries
queries = [
    "What will do on Warning state which device? due to member status on Intertec Perimter FW01?",
    "What is the status of status on Intertec Perimter FW0?",
    "I am not checking"
]

for query in queries:
    print(f"User Query: {query}")
    response = chatbot(query)
    print(f"Chatbot Response: {response}\n")

User Query: What will do on Warning state which device? due to member status on Intertec Perimter FW01?


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
